# Merging into charter and public school files: <br> New CCD data, URLs, overlap-removed webtext, etc.

Authors: Ji Shi, Jaren Haber <br>
Institution: UC Berkeley

Created: Spring 2018 <br>
Last modified: 10-25-2018

In [1]:
import pandas as pd

## Combining CCD 2015-16 data files

In [2]:
#read each component data file
f029 = pd.read_csv("../data/ccd_sch_029_1516_w_2a_011717.csv", encoding = "latin1", low_memory=False)
f033 = pd.read_csv("../data/ccd_sch_033_1516_w_2a_011717.csv", encoding = "latin1", low_memory=False)
f052 = pd.read_csv("../data/ccd_sch_052_1516_w_2a_011717.csv", encoding = "latin1", low_memory=False)
f059 = pd.read_csv("../data/ccd_sch_059_1516_w_2a_011717.csv", encoding = "latin1", low_memory=False)
f129 = pd.read_csv("../data/ccd_sch_129_1516_w_2a_011717.csv", encoding = "latin1", low_memory=False)
fgeo = pd.read_excel("../data/EDGE_GEOCODE_PUBLICSCH_1516/EDGE_GEOCODE_PUBLICSCH_1516.xlsx", encoding = "latin1") #reading excel using the pd.read_excel()

In [3]:
#merge data one by one
#"how" is the way to merge:
#   "outer": keep all the unmatched rows for both side of the merge files
#   "inner": keep no unmatched rows
#   "left": keep the unmatched rows for only the left hand side of the merge file
#   "right": keep the unmatched rows for only the right hand side of the merge file

#"on" is the key that used to match by the one-to-one relationship:
# specify a list of key name for "on"
# if the keys that used are different for the two files, use "left_on" and "right_on"

mergedf = pd.merge(f029,f033, how='outer' , on = ["NCESSCH"])
mergedf = pd.merge(mergedf,f052, how='outer' , on = ["NCESSCH"])
mergedf = pd.merge(mergedf,f059, how='outer' , on = ["NCESSCH"])
mergedf = pd.merge(mergedf,f129, how='outer' , on = ["NCESSCH"])
mergedf = pd.merge(mergedf,fgeo, how='outer' , on = ["NCESSCH"])

In [4]:
mergedf

,SURVYEAR_x,FIPST_x,STABR_x,STATENAME_x,SEANAME_x,LEAID_x,ST_LEAID_x,LEA_NAME_x,SCHID_x,ST_SCHID_x,...,CBSA15,NMCBSA15,CBSATYPE15,CSA15,NMCSA15,NECTA15,NMNECTA15,CD15,SLDL15,SLDU15
0,2015-2016,1.0,AL,ALABAMA,Alabama Department Of Education,100002.0,210,Alabama Youth Services,1667.0,210-0050,...,33860,"Montgomery, AL",1,N,N,N,N,0102,042,030
1,2015-2016,1.0,AL,ALABAMA,Alabama Department Of Education,100002.0,210,Alabama Youth Services,1876.0,210-0001,...,33860,"Montgomery, AL",1,N,N,N,N,0103,075,025
2,2015-2016,1.0,AL,ALABAMA,Alabama Department Of Education,100002.0,210,Alabama Youth Services,1706.0,210-0040,...,13820,"Birmingham-Hoover, AL",1,142,"Birmingham-Hoover-Talladega, AL",N,N,0107,058,020
3,2015-2016,1.0,AL,ALABAMA,Alabama Department Of Education,100002.0,210,Alabama Youth Services,1670.0,210-0060,...,N,N,N,N,N,N,N,0107,068,024
4,2015-2016,1.0,AL,ALABAMA,Alabama Department Of Education,100002.0,210,Alabama Youth Services,277.0,210-0020,...,13820,"Birmingham-Hoover, AL",1,142,"Birmingham-Hoover-Talladega, AL",N,N,0106,044,020
5,2015-2016,1.0,AL,ALABAMA,Alabama Department Of Education,100002.0,210,Alabama Youth Services,1705.0,210-0030,...,33860,"Montgomery, AL",1,N,N,N,N,0103,075,025
6,2015-2016,1.0,AL,ALABAMA,Alabama Department Of Education,100005.0,101,Albertville City,889.0,101-0200,...,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL",N,N,0104,026,009
7,2015-2016,1.0,AL,ALABAMA,Alabama Department Of Education,100005.0,101,Albertville City,1616.0,101-0035,...,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL",N,N,0104,026,009
8,2015-2016,1.0,AL,ALABAMA,Alabama Department Of Education,100005.0,101,Albertville City,870.0,101-0010,...,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL",N,N,0104,026,009
9,2015-2016,1.0,AL,ALABAMA,Alabama Department Of Education,100005.0,101,Albertville City,879.0,101-0110,...,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL",N,N,0104,026,009


In [12]:
mergedf.shape

(102524, 424)

In [6]:
list(mergedf)

['SURVYEAR_x',
 'FIPST_x',
 'STABR_x',
 'STATENAME_x',
 'SEANAME_x',
 'LEAID_x',
 'ST_LEAID_x',
 'LEA_NAME_x',
 'SCHID_x',
 'ST_SCHID_x',
 'NCESSCH',
 'SCH_NAME_x',
 'MSTREET1',
 'MSTREET2',
 'MSTREET3',
 'MCITY',
 'MSTATE',
 'MZIP',
 'MZIP4',
 'PHONE',
 'LSTREET1',
 'LSTREET2',
 'LSTREET3',
 'LCITY_x',
 'LSTATE_x',
 'LZIP_x',
 'LZIP4_x',
 'UNION',
 'OUT_OF_STATE_FLAG',
 'SCH_TYPE_TEXT',
 'SCH_TYPE',
 'RECON_STATUS',
 'GSLO',
 'GSHI',
 'LEVEL',
 'VIRTUAL',
 'BIES',
 'SY_STATUS_TEXT',
 'SY_STATUS',
 'UPDATED_STATUS_TEXT',
 'UPDATED_STATUS',
 'EFFECTIVE_DATE',
 'CHARTER_TEXT',
 'PKOFFERED',
 'KGOFFERED',
 'G1OFFERED',
 'G2OFFERED',
 'G3OFFERED',
 'G4OFFERED',
 'G5OFFERED',
 'G6OFFERED',
 'G7OFFERED',
 'G8OFFERED',
 'G9OFFERED',
 'G10OFFERED',
 'G11OFFERED',
 'G12OFFERED',
 'G13OFFERED',
 'AEOFFERED',
 'UGOFFERED',
 'NOGRADES',
 'CHARTAUTH1',
 'CHARTAUTHN1',
 'CHARTAUTH2',
 'CHARTAUTHN2',
 'IGOFFERED',
 'WEBSITE',
 'SURVYEAR_y',
 'FIPST_y',
 'STABR_y',
 'STATENAME_y',
 'SEANAME_y',
 'LEAI

In [8]:
#show wether there are duplicates or not for the target column of a dataframe

print("Number duplicates: ", sum(mergedf.duplicated(subset='NCESSCH', keep='first')))
print("# entries total: ", len(mergedf['NCESSCH']))
print("# unique entries: ", len(mergedf['NCESSCH'].unique()))

Number duplicates:  0
# entries total:  102524
# unique entries:  102524


In [ ]:
# Define variables to keep from large new data set 'final'
keepvars = ["LAT1516", "LON1516", "AM", "AS", "BL", "HI", "HP", "TR", "TOTFRL", "CHARTER_TEXT", 
            "WEBSITE", "MEMBER", "LEVEL", "LOCALE15", "LEAID"]

In [ ]:
# Reduce merged DF to the columns we want to merge into charter and pubschools DFs
mergedf = mergedf[keepvars]
print(mergedf.shape)
list(mergedf)

## Load, clean, prep other DFs

### All public schools

In [9]:
# Load other data sets into DFs

import gc; gc.disable() # Make it load faster by disabling garbage collector
pubdf = pd.read_pickle("../../nowdata/pubschools_2015.pkl")
charterdf = pd.read_pickle("../../nowdata/charters_2015.pkl")
urldf = pd.read_csv("../../nowdata/backups/charter_URLs_2016.csv", usecols=["NCESSCH", "URL"], low_memory=False)  # Holds URL for each school in 'URL' column
gc.enable() # Re-enable garbage collector for security

print(pubdf.shape)
print(charterdf.shape)
print(urldf.shape)

(137429, 656)
(12877, 680)
(7400, 2)


In [18]:
list(pubdf)

['CMO_NAME',
 'CMO_MEMSUM',
 'SCH_NAME',
 'CMO_STATE',
 'CMO_SCHNUM',
 'CMO_URL',
 'CMO_NUMSTATES',
 'CMO_ALLSTATES',
 'CMO_SECTOR',
 'CMO_NUMSTUDENTS_CREDO17',
 'CMO_TYPE',
 'CMO_WEBTEXT',
 'SURVYEAR',
 'FIPST',
 'STABR',
 'SEANAME',
 'LEAID',
 'ST_LEAID',
 'SCHID',
 'ST_SCHID',
 'NCESSCH',
 'MSTREET1',
 'MSTREET2',
 'MSTREET3',
 'MCITY',
 'MSTATE',
 'MZIP',
 'MZIP4',
 'PHONE',
 'LSTREET1',
 'LSTREET2',
 'LSTREET3',
 'LCITY',
 'LSTATE',
 'LZIP',
 'LZIP4',
 'UNION',
 'OUT_OF_STATE_FLAG',
 'SCH_TYPE_TEXT',
 'SCH_TYPE',
 'RECON_STATUS',
 'GSLO',
 'GSHI',
 'LEVEL',
 'VIRTUAL',
 'BIES',
 'SY_STATUS_TEXT',
 'SY_STATUS',
 'UPDATED_STATUS_TEXT',
 'UPDATED_STATUS',
 'EFFECTIVE_DATE',
 'CHARTER_TEXT',
 'PKOFFERED',
 'KGOFFERED',
 'G1OFFERED',
 'G2OFFERED',
 'G3OFFERED',
 'G4OFFERED',
 'G5OFFERED',
 'G6OFFERED',
 'G7OFFERED',
 'G8OFFERED',
 'G9OFFERED',
 'G10OFFERED',
 'G11OFFERED',
 'G12OFFERED',
 'G13OFFERED',
 'AEOFFERED',
 'UGOFFERED',
 'NOGRADES',
 'CHARTAUTH1',
 'CHARTAUTHN1',
 'CHARTAUTH2

In [26]:
pubdf['PCT_SE_T025_005']

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
10          NaN
11          NaN
12          NaN
13        21.08
14          NaN
15        26.74
16        15.26
17          NaN
18          NaN
19          NaN
20          NaN
21          NaN
22          NaN
23          NaN
24          NaN
25          NaN
26          NaN
27          NaN
28          NaN
29          NaN
          ...  
137399      NaN
137400      NaN
137401      NaN
137402      NaN
137403      NaN
137404      NaN
137405      NaN
137406      NaN
137407      NaN
137408      NaN
137409      NaN
137410      NaN
137411      NaN
137412      NaN
137413      NaN
137414      NaN
137415      NaN
137416      NaN
137417      NaN
137418      NaN
137419      NaN
137420      NaN
137421      NaN
137422      NaN
137423      NaN
137424      NaN
137425      NaN
137426      NaN
137427      NaN
137428      NaN
Name: PCT_SE_T025_005, L

In [27]:
# Generate new columns for public schools data:
pubdf["PCTBA_SD"] = pubdf["PCT_SE_T025_005"] + pubdf["PCT_SE_T025_006"] + pubdf['PCT_SE_T025_007'] + pubdf['PCT_SE_T025_008'] # "% Population 25 Years and Over: Bachelor's Degree or Higher" (school district)
pubdf['PCTETH_SD'] = 1 - pubdf['PCT_SE_T013_002'] # "% Total Population: Nonwhite" (school district)

data_year = int(2015)

pubdf["TOTETH"] = pubdf[["AM", "AS", "BL", "HI", "HP", "TR"]].apply(sum, axis=1) # Number of nonwhite K-12 students
pubdf["PCTETH"] = (pubdf["TOTETH"]/pubdf["MEMBER"]).apply(pd.to_numeric, downcast='float')  # Percent nonwhite K-12 students
pubdf["PCTFRL"] = (pubdf["TOTFRL"]/pubdf["MEMBER"]).apply(pd.to_numeric, downcast='float')  # Percent receiving free/ reduced-price lunch
pubdf["AGE"] = data_year - pubdf["YEAR_OPENED"] # Number of years school has been open

# Recode variables:
pubdf["PLACE"] = pubdf["LOCALE15"].map({11.0:"City", 12.0:"City", 13.0:"City", 21.0:"Suburb", 22.0:"Suburb", 23.0:"Suburb", 31.0:"Town", 32.0:"Town", 33.0:"Town", 41.0:"Rural", 42.0:"Rural", 43.0:"Rural"}).astype('category')
pubdf["LOCALE15"] = pubdf["LOCALE15"].map({11.0:"City (large)", 12.0:"City (midsize)", 13.0:"City (small)", 21.0:"Suburb (large)", 22.0:"Suburb (midsize)", 23.0:"Suburb (small)", 31.0:"Town (fringe)", 32.0:"Town (distant)", 33.0:"Town (remote)", 41.0:"Rural (fringe)", 42.0:"Rural (distant)", 43.0:"Rural (remote)"}).astype('category')
pubdf["TITLEI"] = pubdf["TITLEI"].map({"Yes":1, "No":0}).astype('category')

In [28]:
# Convert matching variable NCESSCH to float type to prevent matching errors
urldf["NCESSCH"] = urldf["NCESSCH"].astype(float)
charterdf["NCESSCH"] = charterdf["NCESSCH"].astype(float) 
pubdf["NCESSCH"] = pubdf["NCESSCH"].astype(float) 

In [29]:
list(pubdf)

['CMO_NAME',
 'CMO_MEMSUM',
 'SCH_NAME',
 'CMO_STATE',
 'CMO_SCHNUM',
 'CMO_URL',
 'CMO_NUMSTATES',
 'CMO_ALLSTATES',
 'CMO_SECTOR',
 'CMO_NUMSTUDENTS_CREDO17',
 'CMO_TYPE',
 'CMO_WEBTEXT',
 'SURVYEAR',
 'FIPST',
 'STABR',
 'SEANAME',
 'LEAID',
 'ST_LEAID',
 'SCHID',
 'ST_SCHID',
 'NCESSCH',
 'MSTREET1',
 'MSTREET2',
 'MSTREET3',
 'MCITY',
 'MSTATE',
 'MZIP',
 'MZIP4',
 'PHONE',
 'LSTREET1',
 'LSTREET2',
 'LSTREET3',
 'LCITY',
 'LSTATE',
 'LZIP',
 'LZIP4',
 'UNION',
 'OUT_OF_STATE_FLAG',
 'SCH_TYPE_TEXT',
 'SCH_TYPE',
 'RECON_STATUS',
 'GSLO',
 'GSHI',
 'LEVEL',
 'VIRTUAL',
 'BIES',
 'SY_STATUS_TEXT',
 'SY_STATUS',
 'UPDATED_STATUS_TEXT',
 'UPDATED_STATUS',
 'EFFECTIVE_DATE',
 'CHARTER_TEXT',
 'PKOFFERED',
 'KGOFFERED',
 'G1OFFERED',
 'G2OFFERED',
 'G3OFFERED',
 'G4OFFERED',
 'G5OFFERED',
 'G6OFFERED',
 'G7OFFERED',
 'G8OFFERED',
 'G9OFFERED',
 'G10OFFERED',
 'G11OFFERED',
 'G12OFFERED',
 'G13OFFERED',
 'AEOFFERED',
 'UGOFFERED',
 'NOGRADES',
 'CHARTAUTH1',
 'CHARTAUTHN1',
 'CHARTAUTH2

In [37]:
pubdf = pd.merge(mergedf,pubdf, how='right' , on = ["NCESSCH"])

In [15]:
print(pubdf.shape)
pubdf

(137429, 656)


,CMO_NAME,CMO_MEMSUM,SCH_NAME,CMO_STATE,CMO_SCHNUM,CMO_URL,CMO_NUMSTATES,CMO_ALLSTATES,CMO_SECTOR,CMO_NUMSTUDENTS_CREDO17,...,POP517_S16,CHILDPOV_S16,TRUE_CHARTER,WEBTEXT,MEMBER,TOTFRL,TITLEI,FTE,YEAR_OPENED,YEAR_CLOSED
0,AAEC,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,NaN,NaN,1,[(https://www.aaechighschools.com/public-chart...,450.0,-1.0,1,-1.00,2010.0,NaN
1,Arizona Agribusiness & Equine Center,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,NaN,NaN,1,[(https://www.aaechighschools.com/public-chart...,450.0,-1.0,1,-1.00,2010.0,NaN
2,AAEC,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,NaN,NaN,1,[(https://www.aaechighschools.com/public-chart...,380.0,282.0,1,-1.00,2005.0,NaN
3,Arizona Agribusiness & Equine Center,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,NaN,NaN,1,[(https://www.aaechighschools.com/public-chart...,380.0,282.0,1,-1.00,2005.0,NaN
4,AAEC,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,NaN,NaN,1,"[(https://www.aaechighschools.com/index.php, F...",386.0,-1.0,1,-1.00,1998.0,NaN
5,Arizona Agribusiness & Equine Center,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,NaN,NaN,1,"[(https://www.aaechighschools.com/index.php, F...",386.0,-1.0,1,-1.00,1998.0,NaN
6,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND ACADEMY MIDDLE,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,NaN,NaN,1,"[(https://ms.cumberlandacademy.com/, False, 0,...",528.0,193.0,1,40.82,2013.0,NaN
7,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND H S,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,NaN,NaN,1,"[(https://hs.cumberlandacademy.com/, False, 0,...",NaN,NaN,NaN,NaN,2015.0,NaN
8,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND ACADEMY,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,NaN,NaN,1,"[(https://elem.cumberlandacademy.com/, False, ...",502.0,204.0,1,36.23,1998.0,NaN
9,ACCELERATED INTERMEDIATE ACADEMY,281.0,ACCELERATED INTERDISCIPLINARY ACAD,TX,3.0,http://www.aiacharterschools.org/,1.0,TX,NaN,NaN,...,NaN,NaN,1,"[(http://www.aiacharterschools.org/, False, 0,...",254.0,243.0,1,16.85,2004.0,NaN


In [38]:
# Save to pickle file
pubdf.to_pickle("../data/pubschools_merged_2015.pkl", index=False)

### All charter schools

In [22]:
list(charterdf)

['NCESSCH',
 'CMO_NAME',
 'CMO_MEMSUM',
 'SCH_NAME',
 'CMO_STATE',
 'CMO_SCHNUM',
 'CMO_URL',
 'CMO_NUMSTATES',
 'CMO_ALLSTATES',
 'CMO_SECTOR',
 'CMO_NUMSTUDENTS_CREDO17',
 'CMO_TYPE',
 'CMO_WEBTEXT',
 'SURVYEAR',
 'FIPST',
 'STABR',
 'SEANAME',
 'LEAID',
 'ST_LEAID',
 'SCHID',
 'ST_SCHID',
 'MSTREET1',
 'MSTREET2',
 'MSTREET3',
 'MCITY',
 'MSTATE',
 'MZIP',
 'MZIP4',
 'PHONE',
 'LSTREET1',
 'LSTREET2',
 'LSTREET3',
 'LCITY',
 'LSTATE',
 'LZIP',
 'LZIP4',
 'UNION',
 'OUT_OF_STATE_FLAG',
 'SCH_TYPE_TEXT',
 'SCH_TYPE',
 'RECON_STATUS',
 'GSLO',
 'GSHI',
 'LEVEL',
 'VIRTUAL',
 'BIES',
 'SY_STATUS_TEXT',
 'SY_STATUS',
 'UPDATED_STATUS_TEXT',
 'UPDATED_STATUS',
 'EFFECTIVE_DATE',
 'CHARTER_TEXT',
 'G13OFFERED',
 'AEOFFERED',
 'UGOFFERED',
 'NOGRADES',
 'CHARTAUTH1',
 'CHARTAUTHN1',
 'CHARTAUTH2',
 'CHARTAUTHN2',
 'IGOFFERED',
 'WEBSITE',
 'FRELCH',
 'REDLCH',
 'AE',
 'TOTAL',
 'AM',
 'AMALM',
 'AMALF',
 'AS',
 'ASALM',
 'ASALF',
 'HI',
 'HIALM',
 'HIALF',
 'BL',
 'BLALM',
 'BLALF',
 'WH',


In [23]:
# Rename ACS columns back to original coding (so STATA can work with them)
# Create tuples for naming scheme: (acs_code, acs_words)
acs_tups = ('FIPS', 'Geo_FIPS'), ('Name of Area', 'Geo_NAME'), ('Qualifying Name', 'Geo_QName'), ('State/U.S.-Abbreviation (USPS)', 'Geo_STUSAB'), ('Summary Level', 'Geo_SUMLEV'), ('Geographic Component', 'Geo_GEOCOMP'), ('File Identification', 'Geo_FILEID'), ('Logical Record Number', 'Geo_LOGRECNO'), ('US', 'Geo_US'), ('Region', 'Geo_REGION'), ('Division', 'Geo_DIVISION'), ('State (Census Code)', 'Geo_STATECE'), ('State (FIPS)', 'Geo_STATE'), ('County', 'Geo_COUNTY'), ('County Subdivision (FIPS)', 'Geo_COUSUB'), ('Place (FIPS Code)', 'Geo_PLACE'), ('Place (State FIPS + Place FIPS)', 'Geo_PLACESE'), ('Census Tract', 'Geo_TRACT'), ('Block Group', 'Geo_BLKGRP'), ('Consolidated City', 'Geo_CONCIT'), ('American Indian Area/Alaska Native Area/Hawaiian Home Land (Census)', 'Geo_AIANHH'), ('American Indian Area/Alaska Native Area/Hawaiian Home Land (FIPS)', 'Geo_AIANHHFP'), ('American Indian Trust Land/Hawaiian Home Land Indicator', 'Geo_AIHHTLI'), ('American Indian Tribal Subdivision (Census)', 'Geo_AITSCE'), ('American Indian Tribal Subdivision (FIPS)', 'Geo_AITS'), ('Alaska Native Regional Corporation (FIPS)', 'Geo_ANRC'), ('Metropolitan and Micropolitan Statistical Area', 'Geo_CBSA'), ('Combined Statistical Area', 'Geo_CSA'), ('Metropolitan Division', 'Geo_METDIV'), ('Metropolitan Area Central City', 'Geo_MACC'), ('Metropolitan/Micropolitan Indicator Flag', 'Geo_MEMI'), ('New England City and Town Combined Statistical Area', 'Geo_NECTA'), ('New England City and Town Area', 'Geo_CNECTA'), ('New England City and Town Area Division', 'Geo_NECTADIV'), ('Urban Area', 'Geo_UA'), ('Urban Area Central Place', 'Geo_UACP'), ('Current Congressional District ***', 'Geo_CDCURR'), ('State Legislative District Upper', 'Geo_SLDU'), ('State Legislative District Lower', 'Geo_SLDL'), ('Voting District', 'Geo_VTD'), ('ZIP Code Tabulation Area (3-digit)', 'Geo_ZCTA3'), ('ZIP Code Tabulation Area (5-digit)', 'Geo_ZCTA5'), ('Subbarrio (FIPS)', 'Geo_SUBMCD'), ('School District (Elementary)', 'Geo_SDELM'), ('School District (Secondary)', 'Geo_SDSEC'), ('School District (Unified)', 'Geo_SDUNI'), ('Urban/Rural', 'Geo_UR'), ('Principal City Indicator', 'Geo_PCI'), ('Traffic Analysis Zone', 'Geo_TAZ'), ('Urban Growth Area', 'Geo_UGA'), ('Public Use Microdata Area - 5% File', 'Geo_PUMA5'), ('Public Use Microdata Area - 1% File', 'Geo_PUMA1'), ('Geographic Identifier', 'Geo_GEOID'), ('Tribal Tract', 'Geo_BTTR'), ('Tribal Block Group', 'Geo_BTBG'), ('Area (Land)', 'Geo_AREALAND'), ('Area (Water)', 'Geo_AREAWATR'), ('Total Population', 'SE_T002_001'), ('Population Density (Per Sq. Mile)', 'SE_T002_002'), ('Area (Land)', 'SE_T002_003'), ('Total Population:', 'SE_T013_001'), ('Total Population: White Alone', 'SE_T013_002'), ('Total Population: Black or African American Alone', 'SE_T013_003'), ('Total Population: American Indian and Alaska Native Alone', 'SE_T013_004'), ('Total Population: Asian Alone', 'SE_T013_005'), ('Total Population: Native Hawaiian and Other Pacific Islander Alone', 'SE_T013_006'), ('Total Population: Some Other Race Alone', 'SE_T013_007'), ('Total Population: Two or More Races', 'SE_T013_008'), ('% Total Population: White Alone', 'PCT_SE_T013_002'), ('% Total Population: Black or African American Alone', 'PCT_SE_T013_003'), ('% Total Population: American Indian and Alaska Native Alone', 'PCT_SE_T013_004'), ('% Total Population: Asian Alone', 'PCT_SE_T013_005'), ('% Total Population: Native Hawaiian and Other Pacific Islander Alone', 'PCT_SE_T013_006'), ('% Total Population: Some Other Race Alone', 'PCT_SE_T013_007'), ('% Total Population: Two or More Races', 'PCT_SE_T013_008'), ('Population 25 Years and Over:', 'SE_T025_001'), ('Population 25 Years and Over: Less than High School', 'SE_T025_002'), ('Population 25 Years and Over: High School Graduate (Includes Equivalency)', 'SE_T025_003'), ('Population 25 Years and Over: Some College', 'SE_T025_004'), ("Population 25 Years and Over: Bachelor's Degree", 'SE_T025_005'), ("Population 25 Years and Over: Master's Degree", 'SE_T025_006'), ('Population 25 Years and Over: Professional School Degree', 'SE_T025_007'), ('Population 25 Years and Over: Doctorate Degree', 'SE_T025_008'), ('% Population 25 Years and Over: Less than High School', 'PCT_SE_T025_002'), ('% Population 25 Years and Over: High School Graduate (Includes Equivalency)', 'PCT_SE_T025_003'), ('% Population 25 Years and Over: Some College', 'PCT_SE_T025_004'), ("% Population 25 Years and Over: Bachelor's Degree", 'PCT_SE_T025_005'), ("% Population 25 Years and Over: Master's Degree", 'PCT_SE_T025_006'), ('% Population 25 Years and Over: Professional School Degree', 'PCT_SE_T025_007'), ('% Population 25 Years and Over: Doctorate Degree', 'PCT_SE_T025_008'), ('Population 3 Years and Over:', 'SE_T028_001'), ('Population 3 Years and Over: Enrolled in School', 'SE_T028_002'), ('Population 3 Years and Over: Not Enrolled in School', 'SE_T028_003'), ('% Population 3 Years and Over: Enrolled in School', 'PCT_SE_T028_002'), ('% Population 3 Years and Over: Not Enrolled in School', 'PCT_SE_T028_003'), ('Civilian Population 16 to 19 Years:', 'SE_T030_001'), ('Civilian Population 16 to 19 Years: Not High School Graduate, Not Enrolled (Dropped Out)', 'SE_T030_002'), ('Civilian Population 16 to 19 Years: High School Graduate, or Enrolled (in School)', 'SE_T030_003'), ('% Civilian Population 16 to 19 Years: Not High School Graduate, Not Enrolled (Dropped Out)', 'PCT_SE_T030_002'), ('% Civilian Population 16 to 19 Years: High School Graduate, or Enrolled (in School)', 'PCT_SE_T030_003'), ('Civilian Population in Labor Force 16 Years and Over:', 'SE_T037_001'), ('Civilian Population in Labor Force 16 Years and Over: Employed', 'SE_T037_002'), ('Civilian Population in Labor Force 16 Years and Over: Unemployed', 'SE_T037_003'), ('% Civilian Population in Labor Force 16 Years and Over: Employed', 'PCT_SE_T037_002'), ('% Civilian Population in Labor Force 16 Years and Over: Unemployed', 'PCT_SE_T037_003'), ('Median Household Income (In 2016 Inflation Adjusted Dollars)', 'SE_T057_001'), ('Gini Index', 'SE_T157_001'), ('Families:', 'SE_T113_001'), ('Families: Income in Below Poverty Level', 'SE_T113_002'), ('Families: Income in Below Poverty Level: Married Couple Family: with Related Child Living  Bellow Poverty Level', 'SE_T113_003'), ('Families: Income in Below Poverty Level: Married Couple Family: No Related Children Under 18 Years', 'SE_T113_004'), ('Families: Income in Below Poverty Level: Male Householder, No Wife Present', 'SE_T113_005'), ('Families: Income in Below Poverty Level: Male Householder, No Wife Present: with Related Children Under 18 Years', 'SE_T113_006'), ('Families: Income in Below Poverty Level: Male Householder, No Wife Present: No Related Children Under 18 Years', 'SE_T113_007'), ('Families: Income in Below Poverty Level: Female Householder, No Husband Present', 'SE_T113_008'), ('Families: Income in Below Poverty Level: Female Householder, No Husband Present: with Related Children Under 18 Years', 'SE_T113_009'), ('Families: Income in Below Poverty Level: Female Householder, No Husband Present: No Related Children Under 18 Years', 'SE_T113_010'), ('Families: Income in at or Above Poverty Level', 'SE_T113_011'), ('% Families: Income in Below Poverty Level', 'PCT_SE_T113_002'), ('% Families: Income in Below Poverty Level: Married Couple Family: with Related Child Living  Bellow Poverty Level', 'PCT_SE_T113_003'), ('% Families: Income in Below Poverty Level: Married Couple Family: No Related Children Under 18 Years', 'PCT_SE_T113_004'), ('% Families: Income in Below Poverty Level: Male Householder, No Wife Present', 'PCT_SE_T113_005'), ('% Families: Income in Below Poverty Level: Male Householder, No Wife Present: with Related Children Under 18 Years', 'PCT_SE_T113_006'), ('% Families: Income in Below Poverty Level: Male Householder, No Wife Present: No Related Children Under 18 Years', 'PCT_SE_T113_007'), ('% Families: Income in Below Poverty Level: Female Householder, No Husband Present', 'PCT_SE_T113_008'), ('% Families: Income in Below Poverty Level: Female Householder, No Husband Present: with Related Children Under 18 Years', 'PCT_SE_T113_009'), ('% Families: Income in Below Poverty Level: Female Householder, No Husband Present: No Related Children Under 18 Years', 'PCT_SE_T113_010'), ('% Families: Income in at or Above Poverty Level', 'PCT_SE_T113_011'), ('Population Under 18 Years of Age for Whom Poverty Status Is Determined:', 'SE_T114_001'), ('Population Under 18 Years of Age for Whom Poverty Status Is Determined: Living in Poverty', 'SE_T114_002'), ('Population Under 18 Years of Age for Whom Poverty Status Is Determined: at or Above Poverty Level', 'SE_T114_003'), ('% Population Under 18 Years of Age for Whom Poverty Status Is Determined: Living in Poverty', 'PCT_SE_T114_002'), ('% Population Under 18 Years of Age for Whom Poverty Status Is Determined: at or Above Poverty Level', 'PCT_SE_T114_003'), ('Total:', 'SE_T130_001'), ('Total: Same House 1 Year Ago', 'SE_T130_002'), ('Total: Moved Within Same County', 'SE_T130_003'), ('Total: Moved From Different County Within Same State', 'SE_T130_004'), ('Total: Moved From Different State', 'SE_T130_005'), ('Total: Moved From Abroad', 'SE_T130_006'), ('% Total: Same House 1 Year Ago', 'PCT_SE_T130_002'), ('% Total: Moved Within Same County', 'PCT_SE_T130_003'), ('% Total: Moved From Different County Within Same State', 'PCT_SE_T130_004'), ('% Total: Moved From Different State', 'PCT_SE_T130_005'), ('% Total: Moved From Abroad', 'PCT_SE_T130_006'), ('Total Population:', 'SE_T133_001'), ('Total Population: Native Born', 'SE_T133_002'), ('Total Population: Foreign Born', 'SE_T133_003'), ('Total Population: Foreign Born: Naturalized Citizen', 'SE_T133_004'), ('Total Population: Foreign Born: Not a Citizen', 'SE_T133_005'), ('% Total Population: Native Born', 'PCT_SE_T133_002'), ('% Total Population: Foreign Born', 'PCT_SE_T133_003'), ('% Total Population: Foreign Born: Naturalized Citizen', 'PCT_SE_T133_004'), ('% Total Population: Foreign Born: Not a Citizen', 'PCT_SE_T133_005'), ('% Total Population: Under 18 Years', 'PCT_SE_T009_002'), ('% Total Population: 18 to 34 Years', 'PCT_SE_T009_003'), ('% Total Population: 35 to 64 Years', 'PCT_SE_T009_004'), ('% Total Population: 65 and Over', 'PCT_SE_T009_005'), ('Population 15 Years and Over:', 'SE_T022_001'), ('% Population 15 Years and Over: Never Married', 'PCT_SE_T022_002'), ('% Population 15 Years and Over: Now Married (Not Including Separated)', 'PCT_SE_T022_003'), ('% Population 15 Years and Over: Separated', 'PCT_SE_T022_004'), ('% Population 15 Years and Over: Widowed', 'PCT_SE_T022_005'), ('% Population 15 Years and Over: Divorced', 'PCT_SE_T022_006'), ('% Population 3 Years and Over Enrolled in School: Public School', 'PCT_SE_T029_002'), ('% Population 3 Years and Over Enrolled in School: Public School: Pre-School', 'PCT_SE_T029_003'), ('% Population 3 Years and Over Enrolled in School: Public School: K-8', 'PCT_SE_T029_004'), ('% Population 3 Years and Over Enrolled in School: Public School: 9-12', 'PCT_SE_T029_005'), ('% Population 3 Years and Over Enrolled in School: Public School: College', 'PCT_SE_T029_006'), ('% Population 3 Years and Over Enrolled in School: Private School', 'PCT_SE_T029_007'), ('% Population 3 Years and Over Enrolled in School: Private School: Pre-School', 'PCT_SE_T029_008'), ('% Population 3 Years and Over Enrolled in School: Private School: K-8', 'PCT_SE_T029_009'), ('% Population 3 Years and Over Enrolled in School: Private School: 9-12', 'PCT_SE_T029_010'), ('% Population 3 Years and Over Enrolled in School: Private School: College', 'PCT_SE_T029_011'), ('% White Alone, Not Hispanic or Latino 16 Years Old  in&nbsp; Civilian Labor Force: Employed', 'PCT_SE_T048_002'), ('% White Alone, Not Hispanic or Latino 16 Years Old  in&nbsp; Civilian Labor Force: Unemployed', 'PCT_SE_T048_003'), ('% Employed Civilian Population 16 Years and Over: Agriculture, Forestry, Fishing and Hunting, and Mining', 'PCT_SE_T049_002'), ('% Employed Civilian Population 16 Years and Over: Construction', 'PCT_SE_T049_003'), ('% Employed Civilian Population 16 Years and Over: Manufacturing', 'PCT_SE_T049_004'), ('% Employed Civilian Population 16 Years and Over: Wholesale Trade', 'PCT_SE_T049_005'), ('% Employed Civilian Population 16 Years and Over: Retail Trade', 'PCT_SE_T049_006'), ('% Employed Civilian Population 16 Years and Over: Transportation and Warehousing, and Utilities', 'PCT_SE_T049_007'), ('% Employed Civilian Population 16 Years and Over: Information', 'PCT_SE_T049_008'), ('% Employed Civilian Population 16 Years and Over: Finance and Insurance, and Real Estate and Rental  and Leasing', 'PCT_SE_T049_009'), ('% Employed Civilian Population 16 Years and Over: Professional, Scientific, and Management, and  Administrative and Waste Management Services', 'PCT_SE_T049_010'), ('% Employed Civilian Population 16 Years and Over: Educational Services, and Health Care and Social  Assistance', 'PCT_SE_T049_011'), ('% Employed Civilian Population 16 Years and Over: Arts, Entertainment, and Recreation, and  Accommodation and Food Services', 'PCT_SE_T049_012'), ('% Employed Civilian Population 16 Years and Over: Other Services, Except Public Administration', 'PCT_SE_T049_013'), ('% Employed Civilian Population 16 Years and Over: Public Administration', 'PCT_SE_T049_014'), ('% Employed Civilian Population 16 Years and Over: Unpaid Family Workers', 'PCT_SE_T053_006'), ('Median Household Income (In 2016 Inflation Adjusted Dollars): White Alone Householder', 'SE_T058_002'), ('Median Household Income (In 2016 Inflation Adjusted Dollars): Black or African American Alone Householder', 'SE_T058_003'), ('Median Household Income (In 2016 Inflation Adjusted Dollars): American Indian and Alaska Native Alone  Householder', 'SE_T058_004'), ('Median Household Income (In 2016 Inflation Adjusted Dollars): Asian Alone', 'SE_T058_005'), ('Median Household Income (In 2016 Inflation Adjusted Dollars): Native Hawaiian and Other Pacific Islander Alone  Householder', 'SE_T058_006'), ('Median Household Income (In 2016 Inflation Adjusted Dollars): Some Other Race Alone Householder', 'SE_T058_007'), ('Median Household Income (In 2016 Inflation Adjusted Dollars): Two or More Races Householder', 'SE_T058_008'), ('Median Household Income (In 2016 Inflation Adjusted Dollars): Hispanic or Latino Householder', 'SE_T058_009'), ('Median Household Income (In 2016 Inflation Adjusted Dollars): White Alone Householder, Not Hispanic or Latino', 'SE_T058_010'), ('Per Capita Income (In 2016 Inflation Adjusted Dollars)', 'SE_T083_001'), ('Median Year Structure Built', 'SE_T098_001'), ('Median Value', 'SE_T101_001'), ('% Renter-Occupied Housing Units: Less than 10 Percent', 'PCT_SE_T103_002'), ('% Renter-Occupied Housing Units: 10 to 29 Percent', 'PCT_SE_T103_003'), ('% Renter-Occupied Housing Units: 30 to 49 Percent', 'PCT_SE_T103_004'), ('% Renter-Occupied Housing Units: 50 Percent or More', 'PCT_SE_T103_005'), ('% Renter-Occupied Housing Units: Not Computed', 'PCT_SE_T103_006'), ('% Foreign-Born Population: Europe', 'PCT_SE_T139_002'), ('% Foreign-Born Population: Asia', 'PCT_SE_T139_034'), ('% Foreign-Born Population: Africa', 'PCT_SE_T139_067'), ('% Foreign-Born Population: Oceania', 'PCT_SE_T139_085'), ('% Foreign-Born Population: Americas', 'PCT_SE_T139_090'), ('Total:', 'SE_T145_001'), ('% Total: No Health Insurance Coverage', 'PCT_SE_T145_002'), ('% Total: with Health Insurance Coverage', 'PCT_SE_T145_003'), ('% Total: with Health Insurance Coverage: Public Health Coverage', 'PCT_SE_T145_004'), ('% Total: with Health Insurance Coverage: Private Health Insurance', 'PCT_SE_T145_005'), ('Occupied Housing Units', 'SE_T165_001'), ('% Occupied Housing Units: Family Households', 'PCT_SE_T165_002'),  ('% Occupied Housing Units: Family Households: Married-Couple Family', 'PCT_SE_T165_003'), ('% Occupied Housing Units: Nonfamily Households', 'PCT_SE_T165_016'), ('% Occupied Housing Units: Nonfamily Households: Householder Living Alone', 'PCT_SE_T165_017'), ('% Occupied Housing Units: With Related Children of the Householder Under 18', 'PCT_SE_T167_002'), ('% Occupied Housing Units: No Related Children of the Householder Under 18', 'PCT_SE_T167_008')
rename_dict = dict((x,y) for x,y in acs_tups)
rename_dict

{'FIPS': 'Geo_FIPS',
 'Name of Area': 'Geo_NAME',
 'Qualifying Name': 'Geo_QName',
 'State/U.S.-Abbreviation (USPS)': 'Geo_STUSAB',
 'Summary Level': 'Geo_SUMLEV',
 'Geographic Component': 'Geo_GEOCOMP',
 'File Identification': 'Geo_FILEID',
 'Logical Record Number': 'Geo_LOGRECNO',
 'US': 'Geo_US',
 'Region': 'Geo_REGION',
 'Division': 'Geo_DIVISION',
 'State (Census Code)': 'Geo_STATECE',
 'State (FIPS)': 'Geo_STATE',
 'County': 'Geo_COUNTY',
 'County Subdivision (FIPS)': 'Geo_COUSUB',
 'Place (FIPS Code)': 'Geo_PLACE',
 'Place (State FIPS + Place FIPS)': 'Geo_PLACESE',
 'Census Tract': 'Geo_TRACT',
 'Block Group': 'Geo_BLKGRP',
 'Consolidated City': 'Geo_CONCIT',
 'American Indian Area/Alaska Native Area/Hawaiian Home Land (Census)': 'Geo_AIANHH',
 'American Indian Area/Alaska Native Area/Hawaiian Home Land (FIPS)': 'Geo_AIANHHFP',
 'American Indian Trust Land/Hawaiian Home Land Indicator': 'Geo_AIHHTLI',
 'American Indian Tribal Subdivision (Census)': 'Geo_AITSCE',
 'American India

In [25]:
# Goal: Rename each instance of acs_words in df_charters to be the instance of acs_code corresponding in acs_tups
charterdf.rename(index=str, columns=rename_dict, inplace=True) # Rename columns using renaming dict
list(charterdf)

['NCESSCH',
 'CMO_NAME',
 'CMO_MEMSUM',
 'SCH_NAME',
 'CMO_STATE',
 'CMO_SCHNUM',
 'CMO_URL',
 'CMO_NUMSTATES',
 'CMO_ALLSTATES',
 'CMO_SECTOR',
 'CMO_NUMSTUDENTS_CREDO17',
 'CMO_TYPE',
 'CMO_WEBTEXT',
 'SURVYEAR',
 'FIPST',
 'STABR',
 'SEANAME',
 'LEAID',
 'ST_LEAID',
 'SCHID',
 'ST_SCHID',
 'MSTREET1',
 'MSTREET2',
 'MSTREET3',
 'MCITY',
 'MSTATE',
 'MZIP',
 'MZIP4',
 'PHONE',
 'LSTREET1',
 'LSTREET2',
 'LSTREET3',
 'LCITY',
 'LSTATE',
 'LZIP',
 'LZIP4',
 'UNION',
 'OUT_OF_STATE_FLAG',
 'SCH_TYPE_TEXT',
 'SCH_TYPE',
 'RECON_STATUS',
 'GSLO',
 'GSHI',
 'LEVEL',
 'VIRTUAL',
 'BIES',
 'SY_STATUS_TEXT',
 'SY_STATUS',
 'UPDATED_STATUS_TEXT',
 'UPDATED_STATUS',
 'EFFECTIVE_DATE',
 'CHARTER_TEXT',
 'G13OFFERED',
 'AEOFFERED',
 'UGOFFERED',
 'NOGRADES',
 'CHARTAUTH1',
 'CHARTAUTHN1',
 'CHARTAUTH2',
 'CHARTAUTHN2',
 'IGOFFERED',
 'WEBSITE',
 'FRELCH',
 'REDLCH',
 'AE',
 'TOTAL',
 'AM',
 'AMALM',
 'AMALF',
 'AS',
 'ASALM',
 'ASALF',
 'HI',
 'HIALM',
 'HIALF',
 'BL',
 'BLALM',
 'BLALF',
 'WH',


In [ ]:
#show wether there are duplicates or not for the target column of a dataframe

print("Number duplicate NCESSCHs in charter data: ", sum(charterdf.duplicated(subset='NCESSCH', keep='first')))
print("# entries total: ", len(charterdf['NCESSCH']))
print("# unique entries: ", len(charterdf['NCESSCH'].unique()))

In [ ]:
#show wether there are duplicates or not for the target column of a dataframe

print("Number duplicate URLs in urldf: ", sum(urldf.duplicated(subset='URL', keep='first')))
print("# entries total: ", len(urldf['URL']))
print("# unique entries: ", len(urldf['URL'].unique()))

In [ ]:
charterdf = pd.merge(urldf, charterdf, how='right', on = ['NCESSCH'])
print(charterdf.shape)
list(charterdf)

In [ ]:
#show wether there are duplicates or not for the target column of a dataframe

print("Number duplicate NCESSCHs in charter data: ", sum(charterdf.duplicated(subset='NCESSCH', keep='first')))
print("# entries total: ", len(charterdf['NCESSCH']))
print("# unique entries: ", len(charterdf['NCESSCH'].unique()))

In [ ]:
#show wether there are duplicates or not for the target column of a dataframe

print("Number duplicates: ", sum(charterdf.duplicated(subset='URL', keep='first')))
print("# entries total: ", len(charterdf['URL']))
print("# unique entries: ", len(charterdf['URL'].unique()))

In [ ]:
# Generate new columns for charter data:
charterdf["DISC_RATIO"] = charterdf["DISCIPLINE_COUNT"]/charterdf["NUMWORDS"]
charterdf["INQ_RATIO"] = charterdf["INQUIRY_COUNT"]/charterdf["NUMWORDS"]
charterdf["ESS_RATIO"] = charterdf["ESS_COUNT"]/charterdf["NUMWORDS"]
charterdf["PROG_RATIO"] = charterdf["PROG_COUNT"]/charterdf["NUMWORDS"]

charterdf["PCTBA_SD"] = charterdf["SE_T025_005"] + charterdf["SE_T025_006"] + charterdf['SE_T025_007'] + charterdf['SE_T025_008'] # % total population with BA or higher (school district)
charterdf['PCTETH_SD'] = 1 - charterdf['SE_T013_002'] # % total population nonwhite/ not white alone (school district)

charterdf["TOTETH"] = charterdf[["AM", "AS", "BL", "HI", "HP", "TR"]].apply(sum, axis=1) # Number of nonwhite K-12 students
charterdf["PCTETH"] = (charterdf["TOTETH"]/charterdf["MEMBER"]).apply(pd.to_numeric, downcast='float')  # Percent nonwhite K-12 students
charterdf["PCTFRL"] = (charterdf["TOTFRL"]/charterdf["MEMBER"]).apply(pd.to_numeric, downcast='float')  # Percent receiving free/ reduced-price lunch

# Recode variables:
charterdf["PLACE"] = charterdf["LOCALE15"].map({11.0:"City", 12.0:"City", 13.0:"City", 21.0:"Suburb", 22.0:"Suburb", 23.0:"Suburb", 31.0:"Town", 32.0:"Town", 33.0:"Town", 41.0:"Rural", 42.0:"Rural", 43.0:"Rural"}).astype('category')
charterdf["LOCALE15"] = charterdf["LOCALE15"].map({11.0:"City (large)", 12.0:"City (midsize)", 13.0:"City (small)", 21.0:"Suburb (large)", 22.0:"Suburb (midsize)", 23.0:"Suburb (small)", 31.0:"Town (fringe)", 32.0:"Town (distant)", 33.0:"Town (remote)", 41.0:"Rural (fringe)", 42.0:"Rural (distant)", 43.0:"Rural (remote)"}).astype('category')
charterdf["TITLEI"] = charterdf["TITLEI"].map({"Yes":1, "No":0}).astype('category')

In [56]:
#delete the duplicate columns with column name end with "_y"

lt = []
for col in final2.columns: 
    if col.endswith("_y"):
        lt.append(col)
final2.drop(lt, axis = 1, inplace=True)


In [74]:
#delete the duplicate columns with column name end with "_x"
lt = {}
for col in final2.columns:
    if col.endswith("_x"):
        lt[col] = col[:-2]
final3 = final2.rename(index=str, columns=lt)
